# Named Entity Recognition using Language Models

This lab will focus on helping you fine-tune Transformers models for Named Entity Recognition. You will use the Datasets library to quickly load and preprocess the WikiNEuRal dataset, getting them ready for training with PyTorch.

## Installation
Let us first install the required libraries

In [ ]:
!pip install datasets transformers seqeval huggingface_hub

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

## Dataset(s)

We will use the Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be done with the functions load_dataset and load_metric.

The notebook should work with any token classification dataset provided by the Datasets library.

In [1]:
from datasets import load_dataset, load_metric, concatenate_datasets
datasets = load_dataset("Babelscape/wikineural")

Extracting data files:   0%|          | 0/27 [00:00<?, ?it/s]

Generating train_pt split: 0 examples [00:00, ? examples/s]

Generating train_it split: 0 examples [00:00, ? examples/s]

Generating val_en split: 0 examples [00:00, ? examples/s]

Generating test_nl split: 0 examples [00:00, ? examples/s]

...

Generating test_en split: 0 examples [00:00, ? examples/s]

Generating test_ru split: 0 examples [00:00, ? examples/s]

Generating test_fr split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/usind/.cache/huggingface/datasets/Babelscape___parquet/Babelscape--wikineural-579d1dc98d2a6b93/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/27 [00:00<?, ?it/s]

The WikiNEuRal dataset has train, validation and test splits for each of the 9 languages.

To access an actual element, you need to select a split first, then give an index:

In [4]:
# datasets["train_en"] refers to the subset of the Wikineural dataset that contains English articles for training. The code datasets["train_en"][0] 
# retrieves the first article in this subset.

datasets["train_en"][0]

{'tokens': ['This',
  'division',
  'also',
  'contains',
  'the',
  'Ventana',
  'Wilderness',
  ',',
  'home',
  'to',
  'the',
  'California',
  'condor',
  '.'],
 'ner_tags': [0, 0, 0, 0, 0, 5, 6, 0, 0, 0, 0, 5, 0, 0],
 'lang': 'en'}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored the following:

In [7]:
# The first line defines a list of NER label strings,
# where each label represents a different named entity category, 
# such as a person (PER), organization (ORG), location (LOC), or miscellaneous (MISC).

# B (Beginning) indicates the first token of a named entity.
# I (Inside) indicates any non-first token of a named entity.
# O (Outside) indicates that a token is not part of a named entity.

label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

#creates a dictionary called labels_vocab where each label string is a key and the corresponding 
#value is an integer that represents that label.

labels_vocab = {'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}

#labels_vocab_reverse dictionary is used to map the integer labels back to their string representations. 
labels_vocab_reverse = {v:k for k,v in labels_vocab.items()}

If you want to train a monolingual model (NER on a single language), you have to use a single language for each split.

In [8]:
train_dataset = datasets["train_en"]
val_dataset = datasets["val_en"]
test_dataset = datasets["test_en"]

This is how you can concatenate multiple dataset splits to train a Multilingual NER model. However, let us focus on Monolingual NER for now.

In [ ]:
# train_dataset = concatenate_datasets([datasets["train_de"], 
#                                       datasets["train_en"], 
#                                       datasets["train_es"], 
#                                       datasets["train_fr"],
#                                       datasets["train_it"],
#                                       datasets["train_nl"],
#                                       datasets["train_pl"],
#                                       datasets["train_pt"],
#                                       datasets["train_ru"]])

# val_dataset = concatenate_datasets([datasets["val_de"], 
#                                     datasets["val_en"], 
#                                     datasets["val_es"], 
#                                     datasets["val_fr"],
#                                     datasets["val_it"],
#                                     datasets["val_nl"],
#                                     datasets["val_pl"],
#                                     datasets["val_pt"],
#                                     datasets["val_ru"]])

# test_dataset = concatenate_datasets([datasets["test_de"], 
#                                     datasets["test_en"], 
#                                     datasets["test_es"], 
#                                     datasets["test_fr"],
#                                     datasets["test_it"],
#                                     datasets["test_nl"],
#                                     datasets["test_pl"],
#                                     datasets["test_pt"],
#                                     datasets["test_ru"]])

## Pre-processing

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

 - we get a tokenizer that corresponds to the model architecture we want to use,
 - we download the vocabulary used when pretraining this specific checkpoint.
 
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") ## We use the BERT tokenizer

C:\Users\usind\anaconda3\envs\nlp2022\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\usind\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [10]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can directly call this tokenizer on one sentence, or you can use the tokenize method to tokenize multiple sentences at once.

In [11]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 8667, 117, 1142, 1110, 1141, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In the above output,

token_type_ids,

is used to distinguish between two sequences that are concatenated together. For example, if we have two sentences "A cat is walking" and "The dog is barking", and we concatenate them as input to a model, then the tokenizer will assign a different token_type_id to the tokens in the first sentence (e.g. 0) and to the tokens in the second sentence (e.g. 1). In the output of the tokenizer, each token is assigned a token_type_id which indicates which sequence it belongs to.

attention_mask,

is used to tell the model which tokens to attend to and which ones to ignore. It is a binary vector that has the same length as the input sequence, where each element is either 0 or 1. The 1s indicate the positions of the tokens that should be attended to by the model, and the 0s indicate the positions of the padding tokens that should be ignored. 


This is a good overal tutorial for preprocessing for Transformer. Check this out to learn more about the preprocessing steps.
https://huggingface.co/transformers/preprocessing.html

Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Just change the value of the following flag:

In [13]:


label_all_tokens = False

We're now ready to write the function that will preprocess our samples. We feed them to the tokenizer with the argument truncation=True (to truncate texts that are bigger than the maximum size allowed by the model) and is_split_into_words=True (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens(i.e.[CLS],[SEP]) have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command.

In [16]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
val_tokenized = val_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/92720 [00:00<?, ? examples/s]

Map:   0%|          | 0/11590 [00:00<?, ? examples/s]

Map:   0%|          | 0/11597 [00:00<?, ? examples/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning

In [17]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label_list), label2id=labels_vocab, id2label=labels_vocab_reverse)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define three more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [39]:
model_name = "bert-base-multilingual-cased" ## You Change this to any generic language model from HuggingFace which supports fine-tuning.
args = TrainingArguments(                   ## This is where all your hyperparameters are, reduce *_batch_size if you are working with a GPU which has limited memory.
    "wikineural-multilingual-ner",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    num_train_epochs=1,                     ## may not be enough for a large data, increase this to 5 later in your experiments.
    weight_decay=0.01,
    push_to_hub=True,
    eval_steps=10000,
    save_steps=10000,
    
    
    #learning_rate is a hyperparameter that controls how quickly the model adapts to the training data. It specifies the size of the step taken at each iteration of the training process in the direction of minimizing the loss. A smaller learning rate results in slower but more precise convergence, while a larger learning rate can lead to faster convergence, but may result in overshooting the optimal solution.
    #per_device_train_batch_size :This parameter specifies how many training examples should be processed together in one forward/backward pass.
    #weight_decay: A regularization term added to the loss function to prevent overfitting. It controls the amount of regularization applied to the weights of the model during training.
    #push_to_hub: A flag that specifies whether to push the trained model to the HuggingFace model hub after training. The model hub is a repository of pre-trained models that can be easily shared and used by the community.
    #eval_steps: The number of training steps between each evaluation of the model on the validation set. This parameter is used to speed up the training process by evaluating the model less frequently.
    #num_train_epochs: The number of times to iterate over the entire training set during training. One epoch means one pass over the entire dataset. Increasing this value may improve the performance of the model, but also increases the training time.



)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate,customize the batch_size and the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [40]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our Trainer is how to compute the metrics from the predictions. Here we will load the seqeval metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.seqeval is a Python library for sequence labeling evaluation. It provides metrics such as precision, recall, and F1-score for named entity recognition (NER) and other sequence labeling tasks. 

In [41]:
metric = load_metric("seqeval")

So we will need to do a bit of post-processing on our predictions:

 - select the predicted index (with the maximum logit) for each token
 - convert it to its string label
 - ignore everywhere we set a label of -100
 
The following function does all this post-processing on the result of Trainer.evaluate (which is a namedtuple containing predictions and labels) before applying the metric:

In [42]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Before you go any further:
1. Login to the HuggigFace CLI on your system using command ``huggingface-cli login``
2. Get your access token from your user account on HuggingFace at https://huggingface.co/settings/tokens (preferably a write access token)
3. Use this token with the command above to ensure being logged in to HuggingFace.

IF you are not working on your own machine with a GPU and have opened this notebook in Colab / Studio Lab, you can use the also cell below to login. You could have used this cell instead of commandline (cli) login, but where is the fun in that?

In [35]:
from huggingface_hub import notebook_login
notebook_login()

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the Trainer:

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

E:\OneDrive - University of Surrey\MODULES\NLP\NLP-2022-main\lab09\wikineural-multilingual-ner is already a clone of https://huggingface.co/Archchana/wikineural-multilingual-ner. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

## Evaluation

The evaluate method allows you to evaluate again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the predict method:

In [ ]:
predictions, labels, _ = trainer.predict(test_tokenized)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

Now, can you try to evaluate the NER task performance on a multilingual dataset? with a different language model?
1. Think about what Language Model on HuggingFace can help you Fine-tune over a multilingual dataset. Hint: XLM-R
2. Go back to the commented out multilingual dataset concatetation notebook cell and try to include a few more languages in the dataset.
3. Try to fine-tune the model on this dataset.
4. Report the performance in a MARKDOWN CELL below, showing your work on multilingual model fine-tuning in the cells below. 
5. Submit this notebook to Google Classroom